---
# Gmsh Tutorials
- this notebook aims to develop my skills using Gmsh Python API
- python-specific reference: https://gmsh.info/doc/texinfo/gmsh.html#x1


In [28]:
# Tutorial 1 - Geometry basics, elementary entities, physical groups
# https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_11_1/tutorials/python/t1.py
import gmsh
import sys

# Before using any functions in the Python API, Gmsh must be initialized:
if not gmsh.is_initialized():
    gmsh.initialize()

# Next we add a new model named "t1":
Name = "test_mesh"
gmsh.model.add(Name)


# 1.POINTS: The first type of `elementary entity' in Gmsh is a `Point'. To create a point
# the Python API function is gmsh.model.geo.addPoint()  or gmsh.model.geo.add_point()
# 3 arguments are the point coordinates (x, y, z)
# - the next (optional) argument is the target mesh size close to the point
# - the last (optional) argument is the point tag/identifier
lc = 1  # target mesh size
z = 0   # fixed z coordinate
gmsh.model.geo.addPoint(0, 0, z, lc, 1)
gmsh.model.geo.addPoint(0, 10, 0, lc, 2)
gmsh.model.geo.addPoint(10, 10, 0, lc, 3)
gmsh.model.geo.addPoint(10, 0, 0, lc, 4)

# 2.LINES: Curves are second type of elementery entities
# Straight line segments: the first 2 arguments are
# - point tags (the start and end points of the line)
# - the last (optional) is the line tag.
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)


# 3.SURFACES: to define a simple rectangular surface, curve loop (ordered list of connected curves)
# has to be defined.  Arguments are
# - list of curve tags
# - the curve loop tag (which must be unique
# amongst curve loops) as the second (optional) argument:
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

# add the surface as a list of curve loops: outline + holes(optional)
gmsh.model.geo.addPlaneSurface([1], 1)


# Before it can be meshed (and, more generally, before it can be used by API
# functions outside of the built-in CAD kernel functions), the CAD entities must
# be synchronized with the Gmsh model
gmsh.model.geo.synchronize()


# physical groups: mathematical ("domain", "boundary"), functional ("left wing", "fuselage") ...
# Here we define a physical curve that groups the left, right and bottom curves
# in a single group (with prescribed tag 5); and a physical surface with name
# "My surface" (with an automatic tag) containing the geometrical surface 1:
gmsh.model.addPhysicalGroup(1, [1, 3, 4], 5)              # curves physical group
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surfaces physical group


# generate a 2D mesh
gmsh.model.mesh.generate(2)


# save it to disk (by default only physical groups are saved)
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")

# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use:
#    gmsh.option.setNumber("Mesh.SaveAll", 1)


# visualize the model in the GUI
gmsh.fltk.run()

# when finished with Gmsh Python API:
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000488143s, CPU 0.000736s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00486965s, CPU 0.004591s)
Info    : 143 nodes 288 elements
Info    : Writing './meshes/test_mesh.msh'...
Info    : Done writing './meshes/test_mesh.msh'
Info    : Writing './meshes/test_mesh.stl'...
Info    : Done writing './meshes/test_mesh.stl'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[cont